In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
# scale = 1/5 
# scale = 1/10
# scale = 1/20 
# scale = 1/40

In [3]:
# def tf_aug(X_in, scaler=scale):
#     X_out = np.zeros_like(X_in)
#     for channel in range(X_in.shape[-1]):
#         this_X = X_in[..., channel][np.newaxis, ..., np.newaxis]
#         scale = np.abs(np.max(this_X) - np.min(this_X)) * scaler
#         this_X = jitter(this_X, sigma=scale)
#         this_X = scaling(this_X, sigma=scale)
#         this_X = time_warp(this_X, sigma=scale)
#         this_X = window_warp(this_X, window_ratio=scale)
#         X_out[..., channel] = this_X[0, ..., 0]
#     return X_out


# def augment_this(X_in, y_in):
#     X_out = tf.numpy_function(tf_aug, [X_in], tf.float32)    
#     return X_out, y_in


In [4]:
X, y, site = load_data()

2022-07-13 09:28:30.821830: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 09:28:30.829015: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 09:28:30.829387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 09:28:30.830429: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
from sklearn.utils import shuffle, resample

In [6]:
from sklearn.metrics import r2_score, median_absolute_error

In [7]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [8]:
n_runs = 10

In [9]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [10]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train = resample(X_train, y_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [11]:
dfs_eval = []
dfs_pred = []

In [ ]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=None)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_1_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_1_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 72)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 100, 6)            1302      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 50, 6)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 16)            304       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 25, 26)            1274      
________________________________

2022-07-13 09:28:38.073035: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-13 09:28:38.613352: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-07-13 09:28:38.896019: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-13 09:28:38.896516: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-13 09:28:38.896587: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2022-07-13 09:28:38.897324: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-13 09:28:38.897511: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx co

10/10 [==============================] - 2s 48ms/step - loss: 71.0605 - mean_squared_error: 71.0605 - rmse: 8.4297 - mean_absolute_error: 7.1774 - val_loss: 17.4874 - val_mean_squared_error: 17.4874 - val_rmse: 4.1818 - val_mean_absolute_error: 3.6381
Epoch 2/1000
10/10 [==============================] - 0s 9ms/step - loss: 28.9110 - mean_squared_error: 28.9110 - rmse: 5.3769 - mean_absolute_error: 4.1905 - val_loss: 21.6802 - val_mean_squared_error: 21.6802 - val_rmse: 4.6562 - val_mean_absolute_error: 3.7207
Epoch 3/1000
10/10 [==============================] - 0s 9ms/step - loss: 23.9348 - mean_squared_error: 23.9348 - rmse: 4.8923 - mean_absolute_error: 3.7683 - val_loss: 8.4862 - val_mean_squared_error: 8.4862 - val_rmse: 2.9131 - val_mean_absolute_error: 2.3470
Epoch 4/1000
10/10 [==============================] - 0s 9ms/step - loss: 20.0618 - mean_squared_error: 20.0618 - rmse: 4.4790 - mean_absolute_error: 3.5583 - val_loss: 11.6000 - val_mean_squared_error: 11.6000 - val_rmse:

In [ ]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=1000)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_2_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_2_pred.csv")

In [ ]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=700)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_3_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_3_pred.csv")

In [ ]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=350)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_4_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_4_pred.csv")

In [ ]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=175)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_5_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_5_pred.csv")

In [ ]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=100)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_6_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_6_pred.csv")

In [ ]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)